### Find the top rooms ignited and the top materials in those rooms that were first ignited

In [13]:
import psycopg2
import pandas as pd
from IPython.display import display

conn = psycopg2.connect(service='nfirs')
pd.options.display.max_rows = 1000

df = pd.read_sql_query("select * from codelookup where fieldid = 'PROP_USE' and length(code_value) = 3 order by code_value", conn)['code_value']
codes = list(df.values)

#### By property use type (batch by property type)

In [17]:
# Create a CSV for each property use type

q = """SELECT x.prop_use,
       area_orig,
       first_ign,
       x.civ_inj,
       x.civ_death,
       x.flame_sprd,
       x.item_sprd, 
       x.cnt
FROM
    ( SELECT *,
             row_number() over (partition BY area_orig
                                ORDER BY area_orig, w.cnt DESC, first_ign, w.flame_sprd,w.item_sprd, w.civ_death, w.civ_inj DESC) row_num
     FROM
         (SELECT distinct bf.area_orig,
                 bf.first_ign,
                 bf.prop_use,
                 bf.flame_sprd,
                 bf.item_sprd,
                 COALESCE(bf.oth_death, 0) as civ_death,
                 COALESCE(bf.oth_inj,0) as civ_inj,
                 count(*) OVER ( PARTITION BY bf.area_orig, bf.first_ign, bf.flame_sprd, bf.item_sprd, COALESCE(bf.oth_death, 0)+COALESCE(bf.oth_inj,0) ) AS cnt,
                 row_number() OVER ( PARTITION BY bf.area_orig, bf.first_ign, bf.flame_sprd, bf.item_sprd, COALESCE(bf.oth_death, 0)+COALESCE(bf.oth_inj,0) ) AS row_numbers
          FROM joint_buildingfires bf
          WHERE bf.area_orig IN
                  ( SELECT area_orig
                   FROM joint_buildingfires
                   WHERE prop_use = %(use)s
                       AND area_orig != 'UU'
                   GROUP BY area_orig
                   ORDER BY count(1) DESC LIMIT 8)
              AND bf.prop_use = %(use)s
              AND bf.first_ign != 'UU'
          ORDER BY area_orig,
                   first_ign ) w
     WHERE w.row_numbers = 1) x
ORDER BY area_orig,
         x.cnt DESC,
         first_ign
"""

# for c in codes[1:2]:
#     df = pd.read_sql_query(q, conn, params=dict(use=c))
#     display(df)

for c in codes:
    df = pd.read_sql_query(q, conn, params=dict(use=c))
    df.to_csv('/tmp/{}.csv'.format(c))

In [4]:
# Testing/sanity checks

q = """SELECT bf.prop_use, bf.area_orig,
                 bf.first_ign,
                 bf.flame_sprd,
                 COALESCE(bf.oth_death, 0) + COALESCE(bf.oth_inj,0) as civ_inj_death,
                 count(*) OVER ( PARTITION BY bf.area_orig, bf.first_ign, bf.flame_sprd, COALESCE(bf.oth_death, 0)+COALESCE(bf.oth_inj,0) ) AS cnt,
                 row_number() OVER ( PARTITION BY bf.area_orig, bf.first_ign, bf.flame_sprd, COALESCE(bf.oth_death, 0)+COALESCE(bf.oth_inj,0) ) AS row_numbers
          FROM buildingfires bf
          WHERE bf.area_orig IN
                  ( SELECT area_orig
                   FROM buildingfires
                   WHERE prop_use = %(use)s
                       AND area_orig != 'UU'
                   GROUP BY area_orig
                   ORDER BY count(1) DESC LIMIT 8)
              AND bf.prop_use = %(use)s
              AND bf.first_ign != 'UU'
          ORDER BY area_orig,
                   first_ign,
                   cnt desc"""

pd.read_sql_query(q, conn, params=dict(use='100'))

DatabaseError: Execution failed on sql 'SELECT bf.prop_use, bf.area_orig,
                 bf.first_ign,
                 bf.flame_sprd,
                 COALESCE(bf.oth_death, 0) + COALESCE(bf.oth_inj,0) as civ_inj_death,
                 count(*) OVER ( PARTITION BY bf.area_orig, bf.first_ign, bf.flame_sprd, COALESCE(bf.oth_death, 0)+COALESCE(bf.oth_inj,0) ) AS cnt,
                 row_number() OVER ( PARTITION BY bf.area_orig, bf.first_ign, bf.flame_sprd, COALESCE(bf.oth_death, 0)+COALESCE(bf.oth_inj,0) ) AS row_numbers
          FROM buildingfires bf
          WHERE bf.area_orig IN
                  ( SELECT area_orig
                   FROM buildingfires
                   WHERE prop_use = %(use)s
                       AND area_orig != 'UU'
                   GROUP BY area_orig
                   ORDER BY count(1) DESC LIMIT 8)
              AND bf.prop_use = %(use)s
              AND bf.first_ign != 'UU'
          ORDER BY area_orig,
                   first_ign,
              

In [5]:
q = """
select count(1)
from joint_buildingfires
where prop_use='100'
    and area_orig = '00'
    and first_ign = '00'
    and COALESCE(oth_death, 0) + COALESCE(oth_inj, 0) = 0
    and flame_sprd = 'N'
"""

pd.read_sql_query(q, conn)

,count
0,26


In [6]:
# Sanity checks

q = """
select area_orig, first_ign, count(1)
from joint_buildingfires
where area_orig != 'UU'
    and first_ign != 'UU'
group by area_orig, first_ign
order by count desc
"""

pd.read_sql_query(q, conn)

,area_orig,first_ign,count
0,24,76,249901
1,76,12,52823
2,24,25,26364
3,74,17,25652
4,21,32,24386
5,75,17,23696
6,24,00,22962
7,24,23,21874
8,21,31,21687
9,24,26,21304


In [7]:
# More sanity checks, including civ death/inj + flame spread

q = """
select area_orig, first_ign, flame_sprd, COALESCE(oth_death, 0)+COALESCE(oth_inj,0) as civ_death_inj, count(1)
from joint_buildingfires
where area_orig != 'UU'
    and first_ign != 'UU'
group by area_orig, first_ign, flame_sprd, civ_death_inj
order by count desc"""

pd.read_sql_query(q, conn)

,area_orig,first_ign,flame_sprd,civ_death_inj,count
0,24,76,N,0,101672
1,24,76,Y,0,69524
2,24,76,,0,60365
3,76,12,N,0,31728
4,76,12,Y,0,20704
5,74,17,N,0,15804
6,75,17,N,0,14695
7,21,32,N,0,14144
8,21,31,N,0,12611
9,24,25,N,0,12584


In [ ]:
# For grouped propety usage only 6 most popular ignition sources

q = """
-- 

SELECT area_orig,
       first_ign,
       x.cnt
FROM
    ( SELECT *,
             row_number() over (partition BY area_orig
                                ORDER BY area_orig, w.cnt DESC, first_ign) row_num
     FROM
         (SELECT bf.area_orig,
                 bf.first_ign,
                 count(*) OVER ( PARTITION BY bf.area_orig, bf.first_ign ) AS cnt,
                 row_number() OVER ( PARTITION BY bf.area_orig, bf.first_ign ) AS row_numbers
          FROM joint_buildingfires bf
          WHERE bf.area_orig IN
                  ( SELECT area_orig
                   FROM joint_buildingfires
                   WHERE prop_use in ('120', '121', '122', '123', '124', '129')
                       AND area_orig != 'UU'
                   GROUP BY area_orig
                   ORDER BY count(1) DESC LIMIT 8)
              AND bf.prop_use in ('120', '121', '122', '123', '124', '129')
              AND bf.first_ign != 'UU'
          ORDER BY area_orig,
                   first_ign ) w
     WHERE w.row_numbers = 1) x
WHERE x.row_num < 7
ORDER BY area_orig,
         x.cnt DESC,
         first_ign
"""

df = pd.read_sql_query(q, conn)
display(df)

In [8]:
# Pull all from buildingfires to CSV

q = """
select prop_use, area_orig, first_ign, oth_inj, oth_death, flame_sprd
from joint_buildingfires"""

df = pd.read_sql_query(q, conn)
df.to_csv('/tmp/buildingfires.csv')